In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['Simhei']  #显示中文
plt.rcParams['axes.unicode_minus']=False    #显示负号 

In [3]:
df=pd.read_excel('Exp1.xlsx')
N_1=df['放射计数'].values
N_2=df['本底计数'].values

In [9]:
print(f'放射性计数的平均值为{N_1.mean():.2f}，样本方差为{np.var(N_1,ddof=1):.2f}')
print(f'放射性计数的平均值为{N_2.mean():.2f}，样本方差为{np.var(N_2,ddof=1):.2f}')

放射性计数的平均值为243.43，样本方差为263.03
放射性计数的平均值为5.06，样本方差为5.26
